In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from preprocess import load_dataset, featurisation
from sklearn.model_selection import train_test_split
from concrete import fhe

/Users/camilziane/.pyenv/versions/3.10.12/envs/hack_hf_zama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from concrete import fhe

# @fhe.compiler({"counter": "encrypted"})
# def increment(counter):
#    return (counter + 1) % 100

# print("Compiling `increment` function")
# increment_fhe = increment.compile(list(range(0, 100)), composable=True)

# print("Generating keyset ...")
# increment_fhe.keygen()

# print("Encrypting the initial counter value")
# counter = 0
# counter_enc = increment_fhe.encrypt(counter)

# print(f"| iteration || decrypted | cleartext |")
# for i in range(10):
#     counter_enc = increment_fhe.run(counter_enc)
#     counter = increment(counter)

#     # For demo purpose; no decryption is needed.
#     counter_dec = increment_fhe.decrypt(counter_enc)
#     print(f"|     {i}     || {counter_dec:<9} | {counter:<9} |")

In [3]:
# from concrete.ml.quantization import QuantizedArray
# X_train.shape
# X_train_q = QuantizedArray(8, X_train)
# X_train_q.qvalues,
# # np.max(X_train), np.min(X_train)

In [4]:
embeddings, labels = load_dataset(
    "./data/lfw_people/George_HW_Bush", cache=True)
# embeddings = featurisation(embeddings)
# standard scale the embeddings
embeddings = (embeddings - np.mean(embeddings, axis=0)) / np.std(embeddings, axis=0)
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

model = LogisticRegression(C=1/5)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Accuracy: {score}")
# recall compute
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)
recall = recall_score(y_test, y_pred, average='macro')
print(f"Recall: {recall}")

Accuracy: 0.9893617021276596
Recall: 0.8333333333333333


In [5]:
model.intercept_

array([-7.28123411])

In [6]:
from concrete.ml.torch.compile import compile_torch_model
import torch
import torch.nn as nn

class RegNet(nn.Module):
    def __init__(self, b):
        super().__init__()
        self.b = nn.Parameter(torch.ones(1)*b)  
    def forward(self, x): 
        X = x[:, :128]
        W = x[:, 128:]
        return ((X @ W.T + self.b) > 0).float()

W = model.coef_
b = model.intercept_.reshape(-1, 1)
reg_net = RegNet(b)
X_stacked = np.hstack([X_train[0].reshape(1,-1), W])

In [7]:
nb_sample = 100
X_train_rand = np.random.normal(0, 1, [nb_sample, X_train.shape[1]])
W_rand = np.random.normal(0, 1, [nb_sample, W.shape[1]])
X_rand_stack = np.hstack([X_train_rand, W_rand])

In [8]:
quantized_module = compile_torch_model(
    reg_net, # our model
    X_rand_stack, # a representative input-set to be used for both quantization and compilation
    n_bits=6,
    rounding_threshold_bits={"n_bits": 6, "method": "approximate"}
)

In [9]:
# duplicate W and b to match the number of samples
W_test = np.repeat(W, X_test.shape[0], axis=0)
X_test_stacked = np.hstack([X_test, W_test])

In [10]:
y_pred = quantized_module.forward(X_test_stacked, fhe="execute")